In [1]:
import graphlab as gl
from graphlab import SArray
import pandas as pd
import numpy as np

In [2]:
sample = gl.SFrame('./sample_submission.csv')
train=gl.SFrame('./train.csv')
test=gl.SFrame('./test.csv')

[INFO] 1452834932 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/howard/anaconda/envs/dato-env/lib/python2.7/site-packages/certifi/cacert.pem
1452834932 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to studiocardo@gmail.com and will expire on November 14, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-89326 - Server binary: /Users/howard/anaconda/envs/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1452834932.log
[INFO] GraphLab Server Version: 1.7.1


PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.175731 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 173836 lines in 0.080186 secs.
PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 2.26565 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,int,str,int,float,float,st

In [3]:
train.num_cols()

299

In [4]:
float(train[train['QuoteConversion_Flag'] == 1].num_rows()) / float(train.num_rows())

0.1875107860695754

In [5]:
train.num_rows()

260753

In [6]:
train.num_cols()

299

In [19]:
train.column_names()

['QuoteNumber',
 'Original_Quote_Date',
 'QuoteConversion_Flag',
 'Field6',
 'Field7',
 'Field8',
 'Field9',
 'Field10',
 'Field11',
 'Field12',
 'CoverageField1A',
 'CoverageField1B',
 'CoverageField2A',
 'CoverageField2B',
 'CoverageField3A',
 'CoverageField3B',
 'CoverageField4A',
 'CoverageField4B',
 'CoverageField5A',
 'CoverageField5B',
 'CoverageField6A',
 'CoverageField6B',
 'CoverageField8',
 'CoverageField9',
 'CoverageField11A',
 'CoverageField11B',
 'SalesField1A',
 'SalesField1B',
 'SalesField2A',
 'SalesField2B',
 'SalesField3',
 'SalesField4',
 'SalesField5',
 'SalesField6',
 'SalesField7',
 'SalesField8',
 'SalesField9',
 'SalesField10',
 'SalesField11',
 'SalesField12',
 'SalesField13',
 'SalesField14',
 'SalesField15',
 'PersonalField1',
 'PersonalField2',
 'PersonalField4A',
 'PersonalField4B',
 'PersonalField5',
 'PersonalField6',
 'PersonalField7',
 'PersonalField8',
 'PersonalField9',
 'PersonalField10A',
 'PersonalField10B',
 'PersonalField11',
 'PersonalField12'

## 1. Data Scrubbing

### 1.1 Assign all NAN to -1

In [7]:
# Make sure you assign it back to the SFrame
#
cols = train.column_names()
for col in cols:
    train = train.fillna(col, -1)

## 1.2 Prepare base models

1. without the result column, QuoteConversion_Flag
2. (1). minus the QuoteNumber
3. (2) minus the Original_Quote_Date, but the Date are reformatted to have sep columns for [weekday, month, year]

### 1.2.1 Create a separate SFrame for the 'QuoteConversion_Flag' column

In [3]:
y = train['QuoteConversion_Flag']

In [ ]:
#
train.remove_column('QuoteConversion_Flag')

## For now, only use columns that are integer

## Drop columns that contain only constant integer values

In [ ]:
train_int = train[int]
train_int.num_cols()  # 267

cols = train_int.column_names()

for col in cols:
    delta = train_int[col][0] - train_int[col].mean()
    std = train_int[col].std()
    
    if ((delta == 0) & (std == 0)):
        #del train[col]
        del train_int[col]

# train_int.num_cols() #265
train.num_cols()

In [ ]:
colnames = train_int.column_names()
for col in colnames:
    max = train_int[col].max()
    train_int[train_int[col].apply (lambda x: x / max if max != 0 else x)]

In [ ]:
train_int.num_columns()

def correlate (column):
    cor = train_int['QuoteConversion_Flag'] * train_int[column]
    return cor.nnz()

features = {}
for col in colnames:
    value =  correlate(col)
    features.update({col: value})

In [ ]:
import operator
sorted_features = sorted(features.items(), key=operator.itemgetter(1))
sorted_features

In [ ]:
train['GeographicField10B']

## Try PCA to see if we can reduce the dimensions

In [ ]:
from matplotlib.mlab import PCA
train_np = train_int.to_numpy()
train_np = numpy.array(train_np)
pca_train = PCA(train_np)

In [ ]:
features = train.column_names()
features.remove('QuoteConversion_Flag')
features

## Let GL Create decide the classification..., we'll start w/ the full set

In [ ]:
model_1 = gl.classifier.create(train, target='QuoteConversion_Flag', features=features)

## Try Boosted Tree with train_int

In [8]:
xbg1 = gl.boosted_trees_classifier.create(train, target='QuoteConversion_Flag')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: WARNING: Detected extremely low variance for feature(s) 'PropertyField6', 'GeographicField10A' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.
PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 247631
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 298
PROGRESS: Number of unpacked features : 298
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   9.069e-01   9.10

In [9]:
top_features = xbg1.get_feature_importance()
top_features

name,index,count
PropertyField29,None,29
PersonalField10A,None,21
PersonalField9,None,17
PersonalField12,None,17
SalesField1B,None,14
SalesField5,None,11
PersonalField25,None,10
PropertyField37,N,10
PropertyField37,Y,8
SalesField1A,None,8


In [11]:
features2 = top_features['name'].unique()
features2 = features2[:14]

In [12]:
#model_2 = gl.classifier.create(train, target='QuoteConversion_Flag', features=features2)
xbg_2 = gl.boosted_trees_classifier.create(train, target='QuoteConversion_Flag', features=features2, max_iterations=3000)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 247769
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 14
PROGRESS: Number of unpacked features : 14
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   8.127e-01   8.126e-01        1.19s
PROGRESS:      1   8.128e-01   8.127e-01        2.40s
PROGRESS:      2   8.128e-01   8.126e-01        3.59s
PROGRESS:      3   8.127e-01   8.125e-01        4.79s
PROGRESS:      4   8.127e-01   8.125e-01        5.98s
PROGRESS:      5   8.128e-01   8.126e-01        7.19s
PROGRESS:      6   8.128e-01   8

In [13]:
xbg_2_results = xbg_2.predict(test)

In [14]:
submit = gl.SFrame('./sample_submission.csv')
submit.remove_column('QuoteConversion_Flag')
submit.add_column(xbg_2_results, name='QuoteConversion_Flag')
submit.export_csv('./xbg_2_3000.csv')

PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.123085 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 173836 lines in 0.101154 secs.


### Peer features 1.

In [15]:
# https://www.kaggle.com/mpearmain/homesite-quote-conversion/xgboost-benchmark/log

features_p1 = ['Field6', 'Field10', 'Field12', 'CoverageField8', 'CoverageField9', 'SalesField7', 'PersonalField7', 'PersonalField16',
'PersonalField17',
'PersonalField18',
'PersonalField19', 
'PropertyField3', 
'PropertyField4', 
'PropertyField5', 
'PropertyField7', 
'PropertyField14', 
'PropertyField28', 
'PropertyField30', 
'PropertyField31', 
'PropertyField32', 
'PropertyField33', 
'PropertyField34', 
'PropertyField36', 
'PropertyField37', 
'PropertyField38', 
'GeographicField63', 
'GeographicField64']

xbg_p1 = gl.boosted_trees_classifier.create(train, target='QuoteConversion_Flag', features=features_p1, max_iterations=3000)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 247617
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 27
PROGRESS: Number of unpacked features : 27
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   8.218e-01   8.203e-01        1.37s
PROGRESS:      1   8.222e-01   8.209e-01        2.81s
PROGRESS:      2   8.224e-01   8.209e-01        4.17s
PROGRESS:      3   8.230e-01   8.220e-01        5.53s
PROGRESS:      4   8.229e-01   8.219e-01        6.93s
PROGRESS:      5   8.229e-01   8.218e-01        8.42s
PROGRESS:      6   8.231e-01   8

In [18]:
xbg_p1.summary()

Class                         : BoostedTreesClassifier

Schema
------
Number of examples            : 247617
Number of classes             : 2
Number of feature columns     : 27
Number of unpacked features   : 27

Settings
--------
Number of trees               : 3000
Max tree depth                : 6
Train accuracy                : 0.9013
Validation accuracy           : 0.8158
Training time (sec)           : 4852.0952



In [16]:
# Test the xbg_p1 model w/ test data
#
xbg_p1_results = xbg_p1.predict(test)


PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.196226 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 173836 lines in 0.120514 secs.


TypeError: Must give column as SArray

In [17]:

# Reformat the test results to conform the submission format
#
submit = gl.SFrame('./sample_submission.csv')
submit.remove_column('QuoteConversion_Flag')
submit.add_column(xbg_p1_results, name='QuoteConversion_Flag')
submit.export_csv('./xbg_p1_3000.csv')

PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.110099 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 173836 lines in 0.078128 secs.


## Model 3

In [ ]:
features3 = top_features['name'][:10]
features3
#model_3 = gl.classifier.create(train, target='QuoteConversion_Flag', features=features3)
model_3 = gl.boosted_trees_classifier.create(train, target='QuoteConversion_Flag', features=features3, max_iterations=200)

In [ ]:
features4 = top_features['name'][:7]
features4
model_4 = gl.classifier.create(train, target='QuoteConversion_Flag', features=features4)

In [ ]:
features5 = ['PersonalField9', 'PropertyField37', 'SalesField5', 'PropertyField29', 'SalesField1A', 'PersonalField1']
model_5 = gl.classifier.create(train, target='QuoteConversion_Flag', features=features5)